### 代码结构
####  Part 1. Generate sample by captcha 数据集获取
####  Part 2. Verify and split the data 数据集划分
####  Part 3. CNN 网络搭建
####  Part 4. Model Training 模型训练
####  Part 5. Model Testing 模型测试

### 目录结构
| 序号 | 文件名称  | 说明         |
| ---- | --------- | ------------ |
| 1    | `captcha_config.json`   | 图片配置文件目录 |
| 2    | `sample_config.json`   | 样本配置文件目录 |
| 3    | `sample/` | 数据集目录   |
| 4    | `model/`  | 模型文件目录 |

* captcha_config.json

{  
  "root_dir": "sample/origin/",　　　　　　　　　　　　　　　　　# 验证码保存路径  
  "image_suffix": "png",　　　　　　　　　　　　　　　　　　　　# 验证码图片后缀  
  "characters": "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ",　　　　# 生成验证码的可选字符  
  "count": 60000,　　　　　　　　　　　　　　　　　　　　　　　# 生成验证码的图片数量  
  "char_count": 4,　　　　　　　　　　　　　　　　　　　　　　# 每张验证码图片上的字符数量  
  "width": 100, 　　　　　　　　　　　　　　　　　　　　　　　# 图片宽度  
  "height": 60　　　　　　　　　　　　　　　　　　　　　　　　# 图片高度  
}  

* sample_config.json

{  
  "origin_image_dir": "sample/origin/",　# 原始文件  
  "new_image_dir": "sample/new_train/",　# 新的训练样本  
  "train_image_dir": "sample/train/",　　# 训练集  
  "test_image_dir": "sample/test/",　　　# 测试集  
  "local_image_dir": "sample/local/",　　# 本地保存图片的路径  
  "model_save_dir": "model/",　　　　　# 从验证码url获取的图片的储存路径  
  "image_width": 100,　　　　　　　　# 图片宽度  
  "image_height": 60,　　　　　　　　　# 图片高度  
  "max_captcha": 4,　　　　　　　　　　# 验证码字符个数  
  "image_suffix": "png",　　　　　　　　# 图片文件后缀  
  "char_set": "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ",　# 验证码识别结果类别  
  "cycle_stop": 60000,　　　　　　　　　　　　　　　　　　　# 启动任务后的训练指定次数后停止  
  "acc_stop": 0.90,　　　　　　　　　　　　　　　　　　　　# 训练到指定准确率后停止  
  "cycle_save": 500,　　　　　　　　　　　　　　　　　　　　# 训练指定次数后定时保存模型  
  "enable_gpu": 0,　　　　　　　　　　　　　　　　　　　　# 是否开启GUP训练  
  "train_batch_size": 128,　　　　　　　　　　　　　　　　# 训练时每次使用的图片张数，如果CPU或者GPU内存太小可以减少这个参数  
  "test_batch_size": 100　　　　　　　　　　　　　　　　　# 每批次测试时验证的图片张数，不要超过验证码集的总数  
}

In [5]:
from captcha.image import ImageCaptcha
import os
import random
import time
import json
from PIL import Image
import shutil
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

### Part 1. Generate sample by captcha

* captcha need to be installed first
* generate 20000 samples

In [5]:
def gen_special_img(text, file_path, width, height):
    # 生成img文件
    generator = ImageCaptcha(width=width, height=height)  # 指定大小，利用captcha生成验证码
    img = generator.generate_image(text)  # 生成图片
    img.save(file_path)  # 保存图片


def gen_ima_by_batch(root_dir, image_suffix, characters, count, char_count, width, height):
    # 判断文件夹是否存在
    if not os.path.exists(root_dir):
        os.makedirs(root_dir)

    for index, i in enumerate(range(count)):
        text = ""
        for j in range(char_count):
            text += random.choice(characters) #随机选择char_count个待定字符，生成验证码

        timec = str(time.time()).replace(".", "") #用时间作为验证码标签的一部分
        p = os.path.join(root_dir, "{}_{}.{}".format(text, timec, image_suffix))
        gen_special_img(text, p, width, height)

        print("Generate captcha image => {}".format(index + 1))


def main():
    with open("captcha_config.json", "r") as f:
        config = json.load(f)
        
    # 配置参数
    root_dir = config["root_dir"]  # 图片储存路径
    image_suffix = config["image_suffix"]  # 图片储存后缀
    characters = config["characters"]  # 图片上显示的字符集 # characters = "0123456789abcdefghijklmnopqrstuvwxyz"
    count = config["count"]  # 生成多少张样本
    char_count = config["char_count"]  # 图片上的字符数量

    # 设置图片高度和宽度
    width = config["width"]
    height = config["height"]

    gen_ima_by_batch(root_dir, image_suffix, characters, count, char_count, width, height)


if __name__ == '__main__':
    main()


Generate captcha image => 1
Generate captcha image => 2
Generate captcha image => 3
Generate captcha image => 4
Generate captcha image => 5
Generate captcha image => 6
Generate captcha image => 7
Generate captcha image => 8
Generate captcha image => 9
Generate captcha image => 10
Generate captcha image => 11
Generate captcha image => 12
Generate captcha image => 13
Generate captcha image => 14
Generate captcha image => 15
Generate captcha image => 16
Generate captcha image => 17
Generate captcha image => 18
Generate captcha image => 19
Generate captcha image => 20
Generate captcha image => 21
Generate captcha image => 22
Generate captcha image => 23
Generate captcha image => 24
Generate captcha image => 25
Generate captcha image => 26
Generate captcha image => 27
Generate captcha image => 28
Generate captcha image => 29
Generate captcha image => 30
Generate captcha image => 31
Generate captcha image => 32
Generate captcha image => 33
Generate captcha image => 34
Generate captcha image 

Generate captcha image => 300
Generate captcha image => 301
Generate captcha image => 302
Generate captcha image => 303
Generate captcha image => 304
Generate captcha image => 305
Generate captcha image => 306
Generate captcha image => 307
Generate captcha image => 308
Generate captcha image => 309
Generate captcha image => 310
Generate captcha image => 311
Generate captcha image => 312
Generate captcha image => 313
Generate captcha image => 314
Generate captcha image => 315
Generate captcha image => 316
Generate captcha image => 317
Generate captcha image => 318
Generate captcha image => 319
Generate captcha image => 320
Generate captcha image => 321
Generate captcha image => 322
Generate captcha image => 323
Generate captcha image => 324
Generate captcha image => 325
Generate captcha image => 326
Generate captcha image => 327
Generate captcha image => 328
Generate captcha image => 329
Generate captcha image => 330
Generate captcha image => 331
Generate captcha image => 332
Generate c

Generate captcha image => 578
Generate captcha image => 579
Generate captcha image => 580
Generate captcha image => 581
Generate captcha image => 582
Generate captcha image => 583
Generate captcha image => 584
Generate captcha image => 585
Generate captcha image => 586
Generate captcha image => 587
Generate captcha image => 588
Generate captcha image => 589
Generate captcha image => 590
Generate captcha image => 591
Generate captcha image => 592
Generate captcha image => 593
Generate captcha image => 594
Generate captcha image => 595
Generate captcha image => 596
Generate captcha image => 597
Generate captcha image => 598
Generate captcha image => 599
Generate captcha image => 600
Generate captcha image => 601
Generate captcha image => 602
Generate captcha image => 603
Generate captcha image => 604
Generate captcha image => 605
Generate captcha image => 606
Generate captcha image => 607
Generate captcha image => 608
Generate captcha image => 609
Generate captcha image => 610
Generate c

Generate captcha image => 855
Generate captcha image => 856
Generate captcha image => 857
Generate captcha image => 858
Generate captcha image => 859
Generate captcha image => 860
Generate captcha image => 861
Generate captcha image => 862
Generate captcha image => 863
Generate captcha image => 864
Generate captcha image => 865
Generate captcha image => 866
Generate captcha image => 867
Generate captcha image => 868
Generate captcha image => 869
Generate captcha image => 870
Generate captcha image => 871
Generate captcha image => 872
Generate captcha image => 873
Generate captcha image => 874
Generate captcha image => 875
Generate captcha image => 876
Generate captcha image => 877
Generate captcha image => 878
Generate captcha image => 879
Generate captcha image => 880
Generate captcha image => 881
Generate captcha image => 882
Generate captcha image => 883
Generate captcha image => 884
Generate captcha image => 885
Generate captcha image => 886
Generate captcha image => 887
Generate c

Generate captcha image => 1128
Generate captcha image => 1129
Generate captcha image => 1130
Generate captcha image => 1131
Generate captcha image => 1132
Generate captcha image => 1133
Generate captcha image => 1134
Generate captcha image => 1135
Generate captcha image => 1136
Generate captcha image => 1137
Generate captcha image => 1138
Generate captcha image => 1139
Generate captcha image => 1140
Generate captcha image => 1141
Generate captcha image => 1142
Generate captcha image => 1143
Generate captcha image => 1144
Generate captcha image => 1145
Generate captcha image => 1146
Generate captcha image => 1147
Generate captcha image => 1148
Generate captcha image => 1149
Generate captcha image => 1150
Generate captcha image => 1151
Generate captcha image => 1152
Generate captcha image => 1153
Generate captcha image => 1154
Generate captcha image => 1155
Generate captcha image => 1156
Generate captcha image => 1157
Generate captcha image => 1158
Generate captcha image => 1159
Generate

...

Generate captcha image => 58889
Generate captcha image => 58890
Generate captcha image => 58891
Generate captcha image => 58892
Generate captcha image => 58893
Generate captcha image => 58894
Generate captcha image => 58895
Generate captcha image => 58896
Generate captcha image => 58897
Generate captcha image => 58898
Generate captcha image => 58899
Generate captcha image => 58900
Generate captcha image => 58901
Generate captcha image => 58902
Generate captcha image => 58903
Generate captcha image => 58904
Generate captcha image => 58905
Generate captcha image => 58906
Generate captcha image => 58907
Generate captcha image => 58908
Generate captcha image => 58909
Generate captcha image => 58910
Generate captcha image => 58911
Generate captcha image => 58912
Generate captcha image => 58913
Generate captcha image => 58914
Generate captcha image => 58915
Generate captcha image => 58916
Generate captcha image => 58917
Generate captcha image => 58918
Generate captcha image => 58919
Generate

Generate captcha image => 59166
Generate captcha image => 59167
Generate captcha image => 59168
Generate captcha image => 59169
Generate captcha image => 59170
Generate captcha image => 59171
Generate captcha image => 59172
Generate captcha image => 59173
Generate captcha image => 59174
Generate captcha image => 59175
Generate captcha image => 59176
Generate captcha image => 59177
Generate captcha image => 59178
Generate captcha image => 59179
Generate captcha image => 59180
Generate captcha image => 59181
Generate captcha image => 59182
Generate captcha image => 59183
Generate captcha image => 59184
Generate captcha image => 59185
Generate captcha image => 59186
Generate captcha image => 59187
Generate captcha image => 59188
Generate captcha image => 59189
Generate captcha image => 59190
Generate captcha image => 59191
Generate captcha image => 59192
Generate captcha image => 59193
Generate captcha image => 59194
Generate captcha image => 59195
Generate captcha image => 59196
Generate

Generate captcha image => 59447
Generate captcha image => 59448
Generate captcha image => 59449
Generate captcha image => 59450
Generate captcha image => 59451
Generate captcha image => 59452
Generate captcha image => 59453
Generate captcha image => 59454
Generate captcha image => 59455
Generate captcha image => 59456
Generate captcha image => 59457
Generate captcha image => 59458
Generate captcha image => 59459
Generate captcha image => 59460
Generate captcha image => 59461
Generate captcha image => 59462
Generate captcha image => 59463
Generate captcha image => 59464
Generate captcha image => 59465
Generate captcha image => 59466
Generate captcha image => 59467
Generate captcha image => 59468
Generate captcha image => 59469
Generate captcha image => 59470
Generate captcha image => 59471
Generate captcha image => 59472
Generate captcha image => 59473
Generate captcha image => 59474
Generate captcha image => 59475
Generate captcha image => 59476
Generate captcha image => 59477
Generate

Generate captcha image => 59709
Generate captcha image => 59710
Generate captcha image => 59711
Generate captcha image => 59712
Generate captcha image => 59713
Generate captcha image => 59714
Generate captcha image => 59715
Generate captcha image => 59716
Generate captcha image => 59717
Generate captcha image => 59718
Generate captcha image => 59719
Generate captcha image => 59720
Generate captcha image => 59721
Generate captcha image => 59722
Generate captcha image => 59723
Generate captcha image => 59724
Generate captcha image => 59725
Generate captcha image => 59726
Generate captcha image => 59727
Generate captcha image => 59728
Generate captcha image => 59729
Generate captcha image => 59730
Generate captcha image => 59731
Generate captcha image => 59732
Generate captcha image => 59733
Generate captcha image => 59734
Generate captcha image => 59735
Generate captcha image => 59736
Generate captcha image => 59737
Generate captcha image => 59738
Generate captcha image => 59739
Generate

Generate captcha image => 59968
Generate captcha image => 59969
Generate captcha image => 59970
Generate captcha image => 59971
Generate captcha image => 59972
Generate captcha image => 59973
Generate captcha image => 59974
Generate captcha image => 59975
Generate captcha image => 59976
Generate captcha image => 59977
Generate captcha image => 59978
Generate captcha image => 59979
Generate captcha image => 59980
Generate captcha image => 59981
Generate captcha image => 59982
Generate captcha image => 59983
Generate captcha image => 59984
Generate captcha image => 59985
Generate captcha image => 59986
Generate captcha image => 59987
Generate captcha image => 59988
Generate captcha image => 59989
Generate captcha image => 59990
Generate captcha image => 59991
Generate captcha image => 59992
Generate captcha image => 59993
Generate captcha image => 59994
Generate captcha image => 59995
Generate captcha image => 59996
Generate captcha image => 59997
Generate captcha image => 59998
Generate

### Part 2. verify and split the data

* PIL(Python Imaging Library)->pillow need to be installed first

In [7]:
def verify(origin_dir, real_width, real_height, image_suffix): #校验图片是否符合设置规范
    
    if not os.path.exists(origin_dir):
        print("[Warning!] Can't find the directory{}，it will be created soon".format(origin_dir))
        os.makedirs(origin_dir)

    print("开始校验原始图片集")
    
    # 图片真实尺寸
    real_size = (real_width, real_height)
    
    # 图片名称列表和数量
    img_list = os.listdir(origin_dir)
    total_count = len(img_list)
    print("原始集共有图片: {}张".format(total_count))

    # 无效图片列表
    bad_img = []

    # 遍历所有图片进行验证
    for index, img_name in enumerate(img_list):
        file_path = os.path.join(origin_dir, img_name)
        # 过滤图片不正确的后缀
        if not img_name.endswith(image_suffix):
            bad_img.append((index, img_name, "文件后缀不正确"))
            continue

        # 过滤图片标签不标准的情况
        prefix, posfix = img_name.split("_")
        if prefix == "" or posfix == "":
            bad_img.append((index, img_name, "图片标签异常"))
            continue

        # 图片无法正常打开
        try:
            img = Image.open(file_path)
        except OSError:
            bad_img.append((index, img_name, "图片无法正常打开"))
            continue

        # 图片尺寸有异常
        if real_size == img.size:
            print("{} pass".format(index), end='\r')
        else:
            bad_img.append((index, img_name, "图片尺寸异常为：{}".format(img.size)))

    print("====以下{}张图片有异常====".format(len(bad_img)))
    if bad_img:
        for b in bad_img:
            print("[第{}张图片] [{}] [{}]".format(b[0], b[1], b[2]))
    else:
        print("未发现异常（共 {} 张图片）".format(len(img_list)))
    print("========end"+'\n')
    return bad_img


def split(origin_dir, train_dir, test_dir, bad_imgs): #分离训练集和测试集
    
    if not os.path.exists(origin_dir):
        print("[Warning!] Can't find the directory{}，it will be created soon".format(origin_dir))
        os.makedirs(origin_dir)

    print("开始分离原始图片集为：测试集（10%）和训练集（90%）")

    # 图片名称列表和数量
    img_list = os.listdir(origin_dir)
    for img in bad_imgs:
        img_list.remove(img)
    total_count = len(img_list)
    print("共分配{}张图片到训练集和测试集，其中{}张为异常留在原始目录".format(total_count, len(bad_imgs)))

    # 创建文件夹
    if not os.path.exists(train_dir):
        os.mkdir(train_dir)

    if not os.path.exists(test_dir):
        os.mkdir(test_dir)

    # 测试集
    test_count = int(total_count*0.10)
    test_set = set()
    for i in range(test_count):
        while True:
            file_name = random.choice(img_list)
            if file_name in test_set:
                pass
            else:
                test_set.add(file_name)
                img_list.remove(file_name)
                break

    test_list = list(test_set)
    print("测试集数量为：{}".format(len(test_list)))
    for file_name in test_list:
        src = os.path.join(origin_dir, file_name)
        dst = os.path.join(test_dir, file_name)
        shutil.move(src, dst)

    # 训练集
    train_list = img_list
    print("训练集数量为：{}".format(len(train_list)))
    for file_name in train_list:
        src = os.path.join(origin_dir, file_name)
        dst = os.path.join(train_dir, file_name)
        shutil.move(src, dst)

    if os.listdir(origin_dir) == 0:
        print("migration done")


def main():
    with open("sample_config.json", "r") as f:
        sample_conf = json.load(f)

    # 图片路径
    origin_dir = sample_conf["origin_image_dir"]
    new_dir = sample_conf["new_image_dir"]
    train_dir = sample_conf["train_image_dir"]
    test_dir = sample_conf["test_image_dir"]
    
    # 图片尺寸
    real_width = sample_conf["image_width"]
    real_height = sample_conf["image_height"]
    
    # 图片后缀
    image_suffix = sample_conf["image_suffix"]

    for image_dir in [origin_dir, new_dir]:
        print('\n' + ">>> 开始校验目录：[{}]".format(image_dir))
        bad_images_info = verify(image_dir, real_width, real_height, image_suffix)
        bad_imgs = []
        for info in bad_images_info:
            bad_imgs.append(info[1])
        split(image_dir, train_dir, test_dir, bad_imgs)


if __name__ == '__main__':
    main()


>>> 开始校验目录：[sample/origin/]
开始校验原始图片集
原始集共有图片: 60000张
====以下0张图片有异常====
未发现异常（共 60000 张图片）
========end

开始分离原始图片集为：测试集（10%）和训练集（90%）
共分配60000张图片到训练集和测试集，其中0张为异常留在原始目录
测试集数量为：6000
训练集数量为：54000

>>> 开始校验目录：[sample/new_train/]
[Warning!] Can't find the directorysample/new_train/，it will be created soon
开始校验原始图片集
原始集共有图片: 0张
====以下0张图片有异常====
未发现异常（共 0 张图片）
========end

开始分离原始图片集为：测试集（10%）和训练集（90%）
共分配0张图片到训练集和测试集，其中0张为异常留在原始目录
测试集数量为：0
训练集数量为：0


### Part 3.  CNN

| Number |               Layers                |
| :--: | :-------------------------------: |
| input |               input               |
|  1   | conv + pool + dropout + ReLU |
|  2   | conv + pool + dropout + ReLU |
|  3   | conv + pool + dropout + ReLU |
|  4   |     FC + dropout + Relu      |
|  5   |         FC + softmax          |
| output |              output               |

In [1]:
class CNN(object):
    def __init__(self, image_height, image_width, max_captcha, char_set, model_save_dir):
        # 初始值
        self.image_height = image_height
        self.image_width = image_width
        self.max_captcha = max_captcha
        self.char_set = char_set
        self.char_set_len = len(char_set)
        self.model_save_dir = model_save_dir  # 模型路径
        with tf.name_scope('parameters'):
            self.w_alpha = 0.01
            self.b_alpha = 0.1
        # tf初始化占位符
        with tf.name_scope('data'):
            self.X = tf.placeholder(tf.float32, [None, self.image_height * self.image_width])  # 特征向量
            self.Y = tf.placeholder(tf.float32, [None, self.max_captcha * self.char_set_len])  # 标签
            self.keep_prob = tf.placeholder(tf.float32)  # dropout值

    @staticmethod
    def convert2gray(img):
        """
        图片转为灰度图，如果是3通道图则计算，单通道图则直接返回
        :param img:
        :return:
        """
        if len(img.shape) > 2:
            r, g, b = img[:, :, 0], img[:, :, 1], img[:, :, 2]
            gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
            return gray
        else:
            return img

    def text2vec(self, text):
        """
        转标签为oneHot编码
        :param text: str
        :return: numpy.array
        """
        text_len = len(text)
        if text_len > self.max_captcha:
            raise ValueError('验证码最长{}个字符'.format(self.max_captcha))

        vector = np.zeros(self.max_captcha * self.char_set_len)

        for i, ch in enumerate(text):
            idx = i * self.char_set_len + self.char_set.index(ch)
            vector[idx] = 1
        return vector

    def model(self):
        x = tf.reshape(self.X, shape=[-1, self.image_height, self.image_width, 1])
        print(">>> input x: {}".format(x))

        # 卷积层1
        wc1 = tf.get_variable(name='wc1', shape=[3, 3, 1, 32], dtype=tf.float32,
                              initializer=tf.contrib.layers.xavier_initializer())
        bc1 = tf.Variable(self.b_alpha * tf.random_normal([32]))
        conv1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(x, wc1, strides=[1, 1, 1, 1], padding='SAME'), bc1))
        conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv1 = tf.nn.dropout(conv1, self.keep_prob)

        # 卷积层2
        wc2 = tf.get_variable(name='wc2', shape=[3, 3, 32, 64], dtype=tf.float32,
                              initializer=tf.contrib.layers.xavier_initializer())
        bc2 = tf.Variable(self.b_alpha * tf.random_normal([64]))
        conv2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv1, wc2, strides=[1, 1, 1, 1], padding='SAME'), bc2))
        conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv2 = tf.nn.dropout(conv2, self.keep_prob)

        # 卷积层3
        wc3 = tf.get_variable(name='wc3', shape=[3, 3, 64, 128], dtype=tf.float32,
                              initializer=tf.contrib.layers.xavier_initializer())
        bc3 = tf.Variable(self.b_alpha * tf.random_normal([128]))
        conv3 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv2, wc3, strides=[1, 1, 1, 1], padding='SAME'), bc3))
        conv3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv3 = tf.nn.dropout(conv3, self.keep_prob)
        print(">>> convolution 3: ", conv3.shape)
        next_shape = conv3.shape[1] * conv3.shape[2] * conv3.shape[3]

        # 全连接层1
        wd1 = tf.get_variable(name='wd1', shape=[next_shape, 1024], dtype=tf.float32,
                              initializer=tf.contrib.layers.xavier_initializer())
        bd1 = tf.Variable(self.b_alpha * tf.random_normal([1024]))
        dense = tf.reshape(conv3, [-1, wd1.get_shape().as_list()[0]])
        dense = tf.nn.relu(tf.add(tf.matmul(dense, wd1), bd1))
        dense = tf.nn.dropout(dense, self.keep_prob)

        # 全连接层2
        wout = tf.get_variable('name', shape=[1024, self.max_captcha * self.char_set_len], dtype=tf.float32,
                               initializer=tf.contrib.layers.xavier_initializer())
        bout = tf.Variable(self.b_alpha * tf.random_normal([self.max_captcha * self.char_set_len]))

        with tf.name_scope('y_prediction'):
            y_predict = tf.add(tf.matmul(dense, wout), bout)

        return y_predict

D:\Anaconda\envs\tensorflow-1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\envs\tensorflow-1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\envs\tensorflow-1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\envs\tensorflow-1.14\lib\site-packages\tensorflow\python\framework\dty

### Part 4. Model Training

In [ ]:
class TrainError(Exception):
    pass


class TrainModel(CNN):
    def __init__(self, train_img_path, verify_img_path, char_set, model_save_dir, cycle_stop, acc_stop, cycle_save,
                 image_suffix, train_batch_size, test_batch_size, verify=False):
        # 训练相关参数
        self.cycle_stop = cycle_stop
        self.acc_stop = acc_stop
        self.cycle_save = cycle_save
        self.train_batch_size = train_batch_size
        self.test_batch_size = test_batch_size

        self.image_suffix = image_suffix
        char_set = [str(i) for i in char_set]

        # 打乱文件顺序+校验图片格式
        self.train_img_path = train_img_path
        self.train_images_list = os.listdir(train_img_path)
        # 校验格式
        if verify:
            self.confirm_image_suffix()
        # 打乱文件顺序
        random.seed(time.time())
        random.shuffle(self.train_images_list)

        # 验证集文件
        self.verify_img_path = verify_img_path
        self.verify_images_list = os.listdir(verify_img_path)

        # 获得图片宽高和字符长度基本信息
        label, captcha_array = self.gen_captcha_text_image(train_img_path, self.train_images_list[0])

        captcha_shape = captcha_array.shape
        captcha_shape_len = len(captcha_shape)
        if captcha_shape_len == 3:
            image_height, image_width, channel = captcha_shape
            self.channel = channel
        elif captcha_shape_len == 2:
            image_height, image_width = captcha_shape
        else:
            raise TrainError("图片转换为矩阵时出错，请检查图片格式")

        # 初始化变量
        super(TrainModel, self).__init__(image_height, image_width, len(label), char_set, model_save_dir)

        # 相关信息打印
        print("-->图片尺寸: {} X {}".format(image_height, image_width))
        print("-->验证码长度: {}".format(self.max_captcha))
        print("-->验证码共{}类 {}".format(self.char_set_len, char_set))
        print("-->使用测试集为 {}".format(train_img_path))
        print("-->使验证集为 {}".format(verify_img_path))

        # test model input and output
        print(">>> Start model test")
        batch_x, batch_y = self.get_batch(0, size=100)
        print(">>> input batch images shape: {}".format(batch_x.shape))
        print(">>> input batch labels shape: {}".format(batch_y.shape))

    @staticmethod
    def gen_captcha_text_image(img_path, img_name):
        """
        返回一个验证码的array形式和对应的字符串标签
        :return:tuple (str, numpy.array)
        """
        # 标签
        label = img_name.split("_")[0]
        # 文件
        img_file = os.path.join(img_path, img_name)
        captcha_image = Image.open(img_file)
        captcha_array = np.array(captcha_image)  # 向量化
        return label, captcha_array

    def get_batch(self, n, size=128):
        batch_x = np.zeros([size, self.image_height * self.image_width])  # 初始化
        batch_y = np.zeros([size, self.max_captcha * self.char_set_len])  # 初始化

        max_batch = int(len(self.train_images_list) / size)
        # print(max_batch)
        if max_batch - 1 < 0:
            raise TrainError("训练集图片数量需要大于每批次训练的图片数量")
        if n > max_batch - 1:
            n = n % max_batch
        s = n * size
        e = (n + 1) * size
        this_batch = self.train_images_list[s:e]
        # print("{}:{}".format(s, e))

        for i, img_name in enumerate(this_batch):
            label, image_array = self.gen_captcha_text_image(self.train_img_path, img_name)
            image_array = self.convert2gray(image_array)  # 灰度化图片
            batch_x[i, :] = image_array.flatten() / 255  # flatten 转为一维
            batch_y[i, :] = self.text2vec(label)  # 生成 oneHot
        return batch_x, batch_y

    def get_verify_batch(self, size=100):
        batch_x = np.zeros([size, self.image_height * self.image_width])  # 初始化
        batch_y = np.zeros([size, self.max_captcha * self.char_set_len])  # 初始化

        verify_images = []
        for i in range(size):
            verify_images.append(random.choice(self.verify_images_list))

        for i, img_name in enumerate(verify_images):
            label, image_array = self.gen_captcha_text_image(self.verify_img_path, img_name)
            image_array = self.convert2gray(image_array)  # 灰度化图片
            batch_x[i, :] = image_array.flatten() / 255  # flatten 转为一维
            batch_y[i, :] = self.text2vec(label)  # 生成 oneHot
        return batch_x, batch_y

    def confirm_image_suffix(self):
        # 在训练前校验所有文件格式
        print("开始校验所有图片后缀")
        for index, img_name in enumerate(self.train_images_list):
            print("{} image pass".format(index), end='\r')
            if not img_name.endswith(self.image_suffix):
                raise TrainError('confirm images suffix：you request [.{}] file but get file [{}]'
                                 .format(self.image_suffix, img_name))
        print("所有图片格式校验通过")

    def train_cnn(self):
        y_predict = self.model()
        print(">>> input batch predict shape: {}".format(y_predict.shape))
        print(">>> End model test")
        # 计算概率 损失
        with tf.name_scope('cost'):
            cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_predict, labels=self.Y))
        # 梯度下降
        with tf.name_scope('train'):
            optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)
        # 计算准确率
        predict = tf.reshape(y_predict, [-1, self.max_captcha, self.char_set_len])  # 预测结果
        max_idx_p = tf.argmax(predict, 2)  # 预测结果
        max_idx_l = tf.argmax(tf.reshape(self.Y, [-1, self.max_captcha, self.char_set_len]), 2)  # 标签
        # 计算准确率
        correct_pred = tf.equal(max_idx_p, max_idx_l)
        with tf.name_scope('char_acc'):
            accuracy_char_count = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        with tf.name_scope('image_acc'):
            accuracy_image_count = tf.reduce_mean(tf.reduce_min(tf.cast(correct_pred, tf.float32), axis=1))
        # 模型保存对象
        saver = tf.train.Saver()
        with tf.Session() as sess:
            init = tf.global_variables_initializer()
            sess.run(init)
            # 恢复模型
            if os.path.exists(self.model_save_dir):
                try:
                    saver.restore(sess, self.model_save_dir)
                # 判断捕获model文件夹中没有模型文件的错误
                except ValueError:
                    print("model文件夹为空，将创建新模型")
            else:
                pass
            # 写入日志
            tf.summary.FileWriter("logs/", sess.graph)

            step = 1
            for i in range(self.cycle_stop):
                batch_x, batch_y = self.get_batch(i, size=self.train_batch_size)
                # 梯度下降训练
                _, cost_ = sess.run([optimizer, cost],
                                    feed_dict={self.X: batch_x, self.Y: batch_y, self.keep_prob: 0.75})
                if step % 10 == 0:
                    # 基于训练集的测试
                    batch_x_test, batch_y_test = self.get_batch(i, size=self.train_batch_size)
                    acc_char = sess.run(accuracy_char_count, feed_dict={self.X: batch_x_test, self.Y: batch_y_test, self.keep_prob: 1.})
                    acc_image = sess.run(accuracy_image_count, feed_dict={self.X: batch_x_test, self.Y: batch_y_test, self.keep_prob: 1.})
                    print("第{}次训练 >>> ".format(step))
                    print("[训练集] 字符准确率为 {:.5f} 图片准确率为 {:.5f} >>> loss {:.10f}".format(acc_char, acc_image, cost_))

                    # with open("loss_train.csv", "a+") as f:
                    #     f.write("{},{},{},{}\n".format(step, acc_char, acc_image, cost_))

                    # 基于验证集的测试
                    batch_x_verify, batch_y_verify = self.get_verify_batch(size=self.test_batch_size)
                    acc_char = sess.run(accuracy_char_count, feed_dict={self.X: batch_x_verify, self.Y: batch_y_verify, self.keep_prob: 1.})
                    acc_image = sess.run(accuracy_image_count, feed_dict={self.X: batch_x_verify, self.Y: batch_y_verify, self.keep_prob: 1.})
                    print("[验证集] 字符准确率为 {:.5f} 图片准确率为 {:.5f} >>> loss {:.10f}".format(acc_char, acc_image, cost_))

                    # with open("loss_test.csv", "a+") as f:
                    #     f.write("{}, {},{},{}\n".format(step, acc_char, acc_image, cost_))

                    # 准确率达到99%后保存并停止
                    if acc_image > self.acc_stop:
                        saver.save(sess, self.model_save_dir)
                        print("验证集准确率达到99%，保存模型成功")
                        break
                # 每训练500轮就保存一次
                if i % self.cycle_save == 0:
                    saver.save(sess, self.model_save_dir)
                    print("定时保存模型成功")
                step += 1
            saver.save(sess, self.model_save_dir)

    def recognize_captcha(self):
        label, captcha_array = self.gen_captcha_text_image(self.train_img_path, random.choice(self.train_images_list))

        f = plt.figure()
        ax = f.add_subplot(111)
        ax.text(0.1, 0.9, "origin:" + label, ha='center', va='center', transform=ax.transAxes)
        plt.imshow(captcha_array)
        # 预测图片
        image = self.convert2gray(captcha_array)
        image = image.flatten() / 255

        y_predict = self.model()

        saver = tf.train.Saver()
        with tf.Session() as sess:
            saver.restore(sess, self.model_save_dir)
            predict = tf.argmax(tf.reshape(y_predict, [-1, self.max_captcha, self.char_set_len]), 2)
            text_list = sess.run(predict, feed_dict={self.X: [image], self.keep_prob: 1.})
            predict_text = text_list[0].tolist()

        print("正确: {}  预测: {}".format(label, predict_text))
        # 显示图片和预测结果
        p_text = ""
        for p in predict_text:
            p_text += str(self.char_set[p])
        print(p_text)
        plt.text(20, 1, 'predict:{}'.format(p_text))
        plt.show()


def main():
    with open("sample_config.json", "r") as f:
        sample_conf = json.load(f)

    train_image_dir = sample_conf["train_image_dir"]
    verify_image_dir = sample_conf["test_image_dir"]
    model_save_dir = sample_conf["model_save_dir"]
    cycle_stop = sample_conf["cycle_stop"]
    acc_stop = sample_conf["acc_stop"]
    cycle_save = sample_conf["cycle_save"]
    enable_gpu = sample_conf["enable_gpu"]
    image_suffix = sample_conf['image_suffix']
    train_batch_size = sample_conf['train_batch_size']
    test_batch_size = sample_conf['test_batch_size']

    char_set = sample_conf["char_set"]

    if not enable_gpu:
        # 设置以下环境变量可开启CPU识别
        os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
        os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

    tm = TrainModel(train_image_dir, verify_image_dir, char_set, model_save_dir, cycle_stop, acc_stop, cycle_save,
                    image_suffix, train_batch_size, test_batch_size, verify=False)
    tm.train_cnn()  # 开始训练模型
    # tm.recognize_captcha()  # 识别图片示例


if __name__ == '__main__':
    main()

-->图片尺寸: 60 X 100
-->验证码长度: 4
-->验证码共62类 ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
-->使用测试集为 sample/train/
-->使验证集为 sample/test/
>>> Start model test
>>> input batch images shape: (100, 6000)
>>> input batch labels shape: (100, 248)
>>> input x: Tensor("Reshape:0", shape=(?, 60, 100, 1), dtype=float32)
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_p

[验证集] 字符准确率为 0.02000 图片准确率为 0.00000 >>> loss 0.0852661654
第520次训练 >>> 
[训练集] 字符准确率为 0.01367 图片准确率为 0.00000 >>> loss 0.0854608491
[验证集] 字符准确率为 0.01750 图片准确率为 0.00000 >>> loss 0.0854608491
第530次训练 >>> 
[训练集] 字符准确率为 0.02930 图片准确率为 0.00000 >>> loss 0.0857636705
[验证集] 字符准确率为 0.01000 图片准确率为 0.00000 >>> loss 0.0857636705
第540次训练 >>> 
[训练集] 字符准确率为 0.00781 图片准确率为 0.00000 >>> loss 0.0856722966
[验证集] 字符准确率为 0.01750 图片准确率为 0.00000 >>> loss 0.0856722966
第550次训练 >>> 
[训练集] 字符准确率为 0.00781 图片准确率为 0.00000 >>> loss 0.0859893486
[验证集] 字符准确率为 0.02000 图片准确率为 0.00000 >>> loss 0.0859893486
第560次训练 >>> 
[训练集] 字符准确率为 0.01367 图片准确率为 0.00000 >>> loss 0.0858059973
[验证集] 字符准确率为 0.02000 图片准确率为 0.00000 >>> loss 0.0858059973
第570次训练 >>> 
[训练集] 字符准确率为 0.00586 图片准确率为 0.00000 >>> loss 0.0860799700
[验证集] 字符准确率为 0.02000 图片准确率为 0.00000 >>> loss 0.0860799700
第580次训练 >>> 
[训练集] 字符准确率为 0.01367 图片准确率为 0.00000 >>> loss 0.0854255781
[验证集] 字符准确率为 0.00250 图片准确率为 0.00000 >>> loss 0.0854255781
第590次训练 >>> 
[训练集] 字符准确率为 0.01562 图片准确率

第1150次训练 >>> 
[训练集] 字符准确率为 0.02148 图片准确率为 0.00000 >>> loss 0.0839486718
[验证集] 字符准确率为 0.02000 图片准确率为 0.00000 >>> loss 0.0839486718
第1160次训练 >>> 
[训练集] 字符准确率为 0.02344 图片准确率为 0.00000 >>> loss 0.0849536955
[验证集] 字符准确率为 0.03000 图片准确率为 0.00000 >>> loss 0.0849536955
第1170次训练 >>> 
[训练集] 字符准确率为 0.01172 图片准确率为 0.00000 >>> loss 0.0842769071
[验证集] 字符准确率为 0.03000 图片准确率为 0.00000 >>> loss 0.0842769071
第1180次训练 >>> 
[训练集] 字符准确率为 0.02539 图片准确率为 0.00000 >>> loss 0.0844449103
[验证集] 字符准确率为 0.01750 图片准确率为 0.00000 >>> loss 0.0844449103
第1190次训练 >>> 
[训练集] 字符准确率为 0.01367 图片准确率为 0.00000 >>> loss 0.0847417787
[验证集] 字符准确率为 0.01250 图片准确率为 0.00000 >>> loss 0.0847417787
第1200次训练 >>> 
[训练集] 字符准确率为 0.01172 图片准确率为 0.00000 >>> loss 0.0844419524
[验证集] 字符准确率为 0.01500 图片准确率为 0.00000 >>> loss 0.0844419524
第1210次训练 >>> 
[训练集] 字符准确率为 0.01172 图片准确率为 0.00000 >>> loss 0.0843240395
[验证集] 字符准确率为 0.01250 图片准确率为 0.00000 >>> loss 0.0843240395
第1220次训练 >>> 
[训练集] 字符准确率为 0.02344 图片准确率为 0.00000 >>> loss 0.0845243409
[验证集] 字符准确率为 0.012

第1780次训练 >>> 
[训练集] 字符准确率为 0.03125 图片准确率为 0.00000 >>> loss 0.0839064345
[验证集] 字符准确率为 0.02250 图片准确率为 0.00000 >>> loss 0.0839064345
第1790次训练 >>> 
[训练集] 字符准确率为 0.01953 图片准确率为 0.00000 >>> loss 0.0835226253
[验证集] 字符准确率为 0.01000 图片准确率为 0.00000 >>> loss 0.0835226253
第1800次训练 >>> 
[训练集] 字符准确率为 0.02148 图片准确率为 0.00000 >>> loss 0.0840266496
[验证集] 字符准确率为 0.03000 图片准确率为 0.00000 >>> loss 0.0840266496
第1810次训练 >>> 
[训练集] 字符准确率为 0.01367 图片准确率为 0.00000 >>> loss 0.0837836191
[验证集] 字符准确率为 0.02250 图片准确率为 0.00000 >>> loss 0.0837836191
第1820次训练 >>> 
[训练集] 字符准确率为 0.01562 图片准确率为 0.00000 >>> loss 0.0839647725
[验证集] 字符准确率为 0.03000 图片准确率为 0.00000 >>> loss 0.0839647725
第1830次训练 >>> 
[训练集] 字符准确率为 0.01172 图片准确率为 0.00000 >>> loss 0.0838179588
[验证集] 字符准确率为 0.02250 图片准确率为 0.00000 >>> loss 0.0838179588
第1840次训练 >>> 
[训练集] 字符准确率为 0.02734 图片准确率为 0.00000 >>> loss 0.0831160992
[验证集] 字符准确率为 0.02500 图片准确率为 0.00000 >>> loss 0.0831160992
第1850次训练 >>> 
[训练集] 字符准确率为 0.01953 图片准确率为 0.00000 >>> loss 0.0833817795
[验证集] 字符准确率为 0.012

第2410次训练 >>> 
[训练集] 字符准确率为 0.04883 图片准确率为 0.00000 >>> loss 0.0822951794
[验证集] 字符准确率为 0.05250 图片准确率为 0.00000 >>> loss 0.0822951794
第2420次训练 >>> 
[训练集] 字符准确率为 0.05273 图片准确率为 0.00000 >>> loss 0.0821183026
[验证集] 字符准确率为 0.04500 图片准确率为 0.00000 >>> loss 0.0821183026
第2430次训练 >>> 
[训练集] 字符准确率为 0.05469 图片准确率为 0.00000 >>> loss 0.0810263529
[验证集] 字符准确率为 0.04000 图片准确率为 0.00000 >>> loss 0.0810263529
第2440次训练 >>> 
[训练集] 字符准确率为 0.06445 图片准确率为 0.00000 >>> loss 0.0816933140
[验证集] 字符准确率为 0.05250 图片准确率为 0.00000 >>> loss 0.0816933140
第2450次训练 >>> 
[训练集] 字符准确率为 0.05078 图片准确率为 0.00000 >>> loss 0.0821291357
[验证集] 字符准确率为 0.05000 图片准确率为 0.00000 >>> loss 0.0821291357
第2460次训练 >>> 
[训练集] 字符准确率为 0.05664 图片准确率为 0.00000 >>> loss 0.0812536553
[验证集] 字符准确率为 0.05000 图片准确率为 0.00000 >>> loss 0.0812536553
第2470次训练 >>> 
[训练集] 字符准确率为 0.06055 图片准确率为 0.00000 >>> loss 0.0812876970
[验证集] 字符准确率为 0.07000 图片准确率为 0.00000 >>> loss 0.0812876970
第2480次训练 >>> 
[训练集] 字符准确率为 0.06055 图片准确率为 0.00000 >>> loss 0.0816058517
[验证集] 字符准确率为 0.047

...

[验证集] 字符准确率为 0.74500 图片准确率为 0.33000 >>> loss 0.0239984319
第13740次训练 >>> 
[训练集] 字符准确率为 0.91602 图片准确率为 0.71875 >>> loss 0.0221478920
[验证集] 字符准确率为 0.78250 图片准确率为 0.33000 >>> loss 0.0221478920
第13750次训练 >>> 
[训练集] 字符准确率为 0.93750 图片准确率为 0.75781 >>> loss 0.0246038251
[验证集] 字符准确率为 0.77500 图片准确率为 0.37000 >>> loss 0.0246038251
第13760次训练 >>> 
[训练集] 字符准确率为 0.93555 图片准确率为 0.77344 >>> loss 0.0227231365
[验证集] 字符准确率为 0.74250 图片准确率为 0.34000 >>> loss 0.0227231365
第13770次训练 >>> 
[训练集] 字符准确率为 0.91797 图片准确率为 0.73438 >>> loss 0.0238020327
[验证集] 字符准确率为 0.77250 图片准确率为 0.37000 >>> loss 0.0238020327
第13780次训练 >>> 
[训练集] 字符准确率为 0.91797 图片准确率为 0.71094 >>> loss 0.0237833112
[验证集] 字符准确率为 0.79500 图片准确率为 0.42000 >>> loss 0.0237833112
第13790次训练 >>> 
[训练集] 字符准确率为 0.90430 图片准确率为 0.67188 >>> loss 0.0232535563
[验证集] 字符准确率为 0.79750 图片准确率为 0.43000 >>> loss 0.0232535563
第13800次训练 >>> 
[训练集] 字符准确率为 0.88867 图片准确率为 0.63281 >>> loss 0.0238628387
[验证集] 字符准确率为 0.82000 图片准确率为 0.45000 >>> loss 0.0238628387
第13810次训练 >>> 
[训练集] 字符准确

[验证集] 字符准确率为 0.80500 图片准确率为 0.41000 >>> loss 0.0219565742
第14370次训练 >>> 
[训练集] 字符准确率为 0.91602 图片准确率为 0.72656 >>> loss 0.0233265888
[验证集] 字符准确率为 0.78250 图片准确率为 0.35000 >>> loss 0.0233265888
第14380次训练 >>> 
[训练集] 字符准确率为 0.94922 图片准确率为 0.82812 >>> loss 0.0215220768
[验证集] 字符准确率为 0.80000 图片准确率为 0.40000 >>> loss 0.0215220768
第14390次训练 >>> 
[训练集] 字符准确率为 0.91211 图片准确率为 0.70312 >>> loss 0.0223535094
[验证集] 字符准确率为 0.77000 图片准确率为 0.36000 >>> loss 0.0223535094
第14400次训练 >>> 
[训练集] 字符准确率为 0.90430 图片准确率为 0.64844 >>> loss 0.0245988835
[验证集] 字符准确率为 0.77250 图片准确率为 0.37000 >>> loss 0.0245988835
第14410次训练 >>> 
[训练集] 字符准确率为 0.91992 图片准确率为 0.70312 >>> loss 0.0239026938
[验证集] 字符准确率为 0.79750 图片准确率为 0.43000 >>> loss 0.0239026938
第14420次训练 >>> 
[训练集] 字符准确率为 0.92773 图片准确率为 0.74219 >>> loss 0.0225291923
[验证集] 字符准确率为 0.81000 图片准确率为 0.40000 >>> loss 0.0225291923
第14430次训练 >>> 
[训练集] 字符准确率为 0.92188 图片准确率为 0.71094 >>> loss 0.0230356026
[验证集] 字符准确率为 0.77250 图片准确率为 0.40000 >>> loss 0.0230356026
第14440次训练 >>> 
[训练集] 字符准确

[验证集] 字符准确率为 0.82250 图片准确率为 0.51000 >>> loss 0.0204185322
第15000次训练 >>> 
[训练集] 字符准确率为 0.93359 图片准确率为 0.75000 >>> loss 0.0227412432
[验证集] 字符准确率为 0.75500 图片准确率为 0.32000 >>> loss 0.0227412432
定时保存模型成功
第15010次训练 >>> 
[训练集] 字符准确率为 0.92969 图片准确率为 0.74219 >>> loss 0.0232769810
[验证集] 字符准确率为 0.80750 图片准确率为 0.35000 >>> loss 0.0232769810
第15020次训练 >>> 
[训练集] 字符准确率为 0.93945 图片准确率为 0.80469 >>> loss 0.0236850902
[验证集] 字符准确率为 0.78000 图片准确率为 0.39000 >>> loss 0.0236850902
第15030次训练 >>> 
[训练集] 字符准确率为 0.92773 图片准确率为 0.76562 >>> loss 0.0204239059
[验证集] 字符准确率为 0.76750 图片准确率为 0.36000 >>> loss 0.0204239059
第15040次训练 >>> 
[训练集] 字符准确率为 0.94141 图片准确率为 0.79688 >>> loss 0.0208542235
[验证集] 字符准确率为 0.77250 图片准确率为 0.37000 >>> loss 0.0208542235
第15050次训练 >>> 
[训练集] 字符准确率为 0.94531 图片准确率为 0.79688 >>> loss 0.0201268196
[验证集] 字符准确率为 0.80500 图片准确率为 0.39000 >>> loss 0.0201268196
第15060次训练 >>> 
[训练集] 字符准确率为 0.93945 图片准确率为 0.80469 >>> loss 0.0198570676
[验证集] 字符准确率为 0.78500 图片准确率为 0.38000 >>> loss 0.0198570676
第15070次训练 >>> 
[

第15620次训练 >>> 
[训练集] 字符准确率为 0.93750 图片准确率为 0.80469 >>> loss 0.0209319443
[验证集] 字符准确率为 0.79000 图片准确率为 0.38000 >>> loss 0.0209319443
第15630次训练 >>> 
[训练集] 字符准确率为 0.93945 图片准确率为 0.79688 >>> loss 0.0209432449
[验证集] 字符准确率为 0.78250 图片准确率为 0.37000 >>> loss 0.0209432449
第15640次训练 >>> 
[训练集] 字符准确率为 0.94141 图片准确率为 0.78906 >>> loss 0.0203429330
[验证集] 字符准确率为 0.78750 图片准确率为 0.41000 >>> loss 0.0203429330
第15650次训练 >>> 
[训练集] 字符准确率为 0.95508 图片准确率为 0.82812 >>> loss 0.0208666231
[验证集] 字符准确率为 0.75250 图片准确率为 0.34000 >>> loss 0.0208666231
第15660次训练 >>> 
[训练集] 字符准确率为 0.94922 图片准确率为 0.82031 >>> loss 0.0206855144
[验证集] 字符准确率为 0.76000 图片准确率为 0.30000 >>> loss 0.0206855144
第15670次训练 >>> 
[训练集] 字符准确率为 0.94727 图片准确率为 0.79688 >>> loss 0.0193795525
[验证集] 字符准确率为 0.79750 图片准确率为 0.43000 >>> loss 0.0193795525
第15680次训练 >>> 
[训练集] 字符准确率为 0.94922 图片准确率为 0.82031 >>> loss 0.0211569238
[验证集] 字符准确率为 0.76750 图片准确率为 0.38000 >>> loss 0.0211569238
第15690次训练 >>> 
[训练集] 字符准确率为 0.94336 图片准确率为 0.79688 >>> loss 0.0214117970
[验证集] 字符准确

[验证集] 字符准确率为 0.77250 图片准确率为 0.37000 >>> loss 0.0198398922
第16250次训练 >>> 
[训练集] 字符准确率为 0.93750 图片准确率为 0.77344 >>> loss 0.0211185869
[验证集] 字符准确率为 0.76750 图片准确率为 0.38000 >>> loss 0.0211185869
第16260次训练 >>> 
[训练集] 字符准确率为 0.96289 图片准确率为 0.85938 >>> loss 0.0189263131
[验证集] 字符准确率为 0.79250 图片准确率为 0.40000 >>> loss 0.0189263131
第16270次训练 >>> 
[训练集] 字符准确率为 0.95312 图片准确率为 0.82031 >>> loss 0.0199143123
[验证集] 字符准确率为 0.79750 图片准确率为 0.43000 >>> loss 0.0199143123
第16280次训练 >>> 
[训练集] 字符准确率为 0.92773 图片准确率为 0.75781 >>> loss 0.0200252905
[验证集] 字符准确率为 0.77500 图片准确率为 0.38000 >>> loss 0.0200252905
第16290次训练 >>> 
[训练集] 字符准确率为 0.95117 图片准确率为 0.82812 >>> loss 0.0185988061
[验证集] 字符准确率为 0.77000 图片准确率为 0.36000 >>> loss 0.0185988061
第16300次训练 >>> 
[训练集] 字符准确率为 0.95312 图片准确率为 0.82812 >>> loss 0.0186024345
[验证集] 字符准确率为 0.80500 图片准确率为 0.46000 >>> loss 0.0186024345
第16310次训练 >>> 
[训练集] 字符准确率为 0.95898 图片准确率为 0.85156 >>> loss 0.0196859986
[验证集] 字符准确率为 0.76000 图片准确率为 0.35000 >>> loss 0.0196859986
第16320次训练 >>> 
[训练集] 字符准确

### Part 5. Model Testing

In [6]:
class TestError(Exception):
    pass


class TestBatch(CNN):
    def __init__(self, img_path, char_set, model_save_dir, total):
        # 模型路径
        self.model_save_dir = model_save_dir
        # 打乱文件顺序
        self.img_path = img_path
        self.img_list = os.listdir(img_path)
        random.seed(time.time())
        random.shuffle(self.img_list)

        # 重新获取验证集 —— 图片宽高和字符长度基本信息
        label, captcha_array = self.gen_captcha_text_image()

        captcha_shape = captcha_array.shape
        captcha_shape_len = len(captcha_shape)
        if captcha_shape_len == 3:
            image_height, image_width, channel = captcha_shape
            self.channel = channel
        elif captcha_shape_len == 2:
            image_height, image_width = captcha_shape
        else:
            raise TestError("图片转换为矩阵时出错，请检查图片格式")

        # 初始化变量
        super(TestBatch, self).__init__(image_height, image_width, len(label), char_set, model_save_dir)
        self.total = total

        # 相关信息打印
        print("-->图片尺寸: {} X {}".format(image_height, image_width))
        print("-->验证码长度: {}".format(self.max_captcha))
        print("-->验证码共{}类 {}".format(self.char_set_len, char_set))
        print("-->使用测试集为 {}".format(img_path))

    def gen_captcha_text_image(self):
        """
        返回一个验证码的array形式和对应的字符串标签
        :return:tuple (str, numpy.array)
        """
        img_name = random.choice(self.img_list)
        
        # 标签
        label = img_name.split("_")[0]
        
        # 文件
        img_file = os.path.join(self.img_path, img_name)
        captcha_image = Image.open(img_file)
        captcha_array = np.array(captcha_image)  # 向量化

        return label, captcha_array

    def test_batch(self):
        y_predict = self.model()
        total = self.total
        right = 0
        saver = tf.train.Saver()
        with tf.Session() as sess:
            saver.restore(sess, self.model_save_dir)
            s = time.time()
            for i in range(total):
                test_text, test_image = self.gen_captcha_text_image()  # 随机获取验证集
                test_image = self.convert2gray(test_image)
                test_image = test_image.flatten() / 255

                predict = tf.argmax(tf.reshape(y_predict, [-1, self.max_captcha, self.char_set_len]), 2)
                text_list = sess.run(predict, feed_dict={self.X: [test_image], self.keep_prob: 1.})
                predict_text = text_list[0].tolist()
                p_text = ""
                
                for p in predict_text:
                    p_text += str(self.char_set[p])
                print("origin: {} predict: {}".format(test_text, p_text))
                if test_text == p_text:
                    right += 1
                else:
                    pass
            e = time.time()
        rate = str(right/total * 100) + "%"
        print("测试结果： {}/{}".format(right, total))
        print("{}个样本识别耗时{}秒，准确率{}".format(total, e-s, rate))


def main():
    with open("sample_config.json", "r") as f:
        sample_conf = json.load(f)

    test_image_dir = sample_conf["test_image_dir"]
    model_save_dir = sample_conf["model_save_dir"]
    char_set = sample_conf["char_set"]

    total = 1000
    tb = TestBatch(test_image_dir, char_set, model_save_dir, total)
    tb.test_batch()


if __name__ == '__main__':
    main()

-->图片尺寸: 60 X 100
-->验证码长度: 4
-->验证码共62类 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
-->使用测试集为 sample/test/
>>> input x: Tensor("Reshape:0", shape=(?, 60, 100, 1), dtype=float32)
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
>>> convolution 3:  (?, 8, 13, 128)
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model/
origin: Dm34 predict: Dm34
origin: rJqj predict: rJqj
origin: uDQT predict: uOQT
origin: 49nz predict: 49nz
origin: UEf9 predict: UYf9
origin: BQqH pre

origin: z0we predict: Z0we
origin: jTgt predict: jIAt
origin: iOIf predict: ioZf
origin: mCSc predict: mC9c
origin: i7As predict: iAAs
origin: BbPs predict: BdPs
origin: y2Z3 predict: y2Z3
origin: acEU predict: aoEU
origin: RRNf predict: RRYf
origin: dx9O predict: dx9O
origin: hFD2 predict: hFo2
origin: NqaK predict: NqaK
origin: gfpL predict: gfpL
origin: 0C1g predict: 0Czg
origin: Lku7 predict: LMu7
origin: fK7U predict: fK7U
origin: onbD predict: onbO
origin: zuMl predict: zzMl
origin: VRVU predict: 4PVU
origin: gM2s predict: gxZs
origin: PpFJ predict: RPFJ
origin: jMeS predict: jNeS
origin: mrgk predict: mrgk
origin: GHfI predict: GHfI
origin: gwc0 predict: gwcm
origin: hDF9 predict: hGF9
origin: R3Pl predict: R3PU
origin: gx6h predict: gx6h
origin: Y4XW predict: Y4AW
origin: BN5m predict: BNSm
origin: n3Ni predict: naNi
origin: 8PvY predict: 8PvY
origin: 2LGu predict: 2LGu
origin: FCgD predict: Fo8D
origin: OnDn predict: OnDn
origin: n6lc predict: n6lc
origin: 7fDs predict: 7fQs
o

origin: eTlD predict: eTlO
origin: pU7Y predict: pUOY
origin: Myco predict: Nyco
origin: 4BVD predict: 4BV0
origin: 4gj0 predict: 4gjD
origin: HRFy predict: HPFy
origin: 9zRr predict: 9zRr
origin: CzY9 predict: CZY9
origin: wtTD predict: 4tTD
origin: WvQp predict: vVQQ
origin: z6AE predict: a6AE
origin: LKjK predict: LKjK
origin: 4hVG predict: 4hvG
origin: T2z7 predict: TZz7
origin: ITGT predict: ITGT
origin: mFdj predict: mFFj
origin: tLhH predict: tLhH
origin: 27Ty predict: 2l7Y
origin: Ou5t predict: Ou6t
origin: 1EsG predict: 1csG
origin: vSOW predict: VG0W
origin: cFVJ predict: cFVJ
origin: VN8R predict: vNBR
origin: cxeS predict: cKeT
origin: 4NaO predict: 4NaD
origin: XJkN predict: xJkN
origin: uAn3 predict: uAn3
origin: ZQPb predict: ZQPb
origin: 9rMS predict: 9rMS
origin: TODj predict: TO0j
origin: 33yu predict: 33yu
origin: FI57 predict: FI5T
origin: Eg8d predict: Eg8d
origin: IzvC predict: IzVC
origin: v1LB predict: v11B
origin: WpcP predict: WRcP
origin: 4LJV predict: 4LJV
o

* 5-s-S
* 1-l-I
* 8-B
* 0-o-O
* c-C u-U v-V p-P w-W z-Z x-X 